# Implémentation de ISTA
---
**Aymen.B**

Je me repose sur l'article *"A Fast Iterative Shrinkage-Thresholding Algorithm for Linear Inverse Problems"* de **Amir Beck** et **Marc Teboulle**

## Formulation du problème

On cherche à résoudre un problème de cette forme :

$$\min_{x\in\mathbb{R}^n}F(x):=f(x)+g(x)$$

- $f(x)$ doit être convexe, différentiable et de gradient lipschitz continu
- $g(x)$ pas forcément convexe, pas forcément différentiable, proximable

Une fonction différentiable $f : \mathbb{R}^n \to \mathbb{R}$ a un gradient $L$-Lipschitz-continu s’il existe une constante $L > 0$ telle que :
$$\|\nabla f(x)-\nabla f(y)\|_2\leq L\|x-y\|_2,\quad\forall x,y\in\mathbb{R}^n.$$

Une fonction $g : \mathbb{R}^n \to \mathbb{R} \cup {+\infty}$ est dite proximable si pour tout $z \in \mathbb{R}^n$ et tout $\gamma > 0$, on peut calculer cette opérateur :
$$\mathrm{prox}_{\gamma g}(z):=\arg\min_{x\in\mathbb{R}^n}\left\{\frac{1}{2\gamma}\|x-z\|^2+g(x)\right\}.$$

# ISTA (Iterative Shrinkage-Thresholding Algorithm)

## Théorie

On commence par approximer quadratiquement $F(x)$ par un opérateur $Q_L(x, y)$ défini par qu'on obtiens par un dévellopement de taylor et la propriété de lipschitzienité :  
$$Q_L(\mathbf{x},\mathbf{y}) := f(\mathbf{y}) + \langle \mathbf{x} - \mathbf{y}, \nabla f(\mathbf{y}) \rangle + \frac{L}{2}\|\mathbf{x} - \mathbf{y}\|^2 + g(\mathbf{x}) \quad \text{(cf. lemmes 2.1, 2.2 et 2.3)}$$

On choisit ici le plus grand pas autorisé pour garantir l'approximation ci-dessus, c'est-à-dire $t = \frac{1}{L}$, où $L \geq L(f)$ avec $L(f)$ la constante de Lipschitz du gradient $\nabla f$.

Cette fonction $Q_L(x, y)$ admet un unique minimum pour $\mathbf{y}$ fixé, noté $p_L(\mathbf{y})$, défini par :  
$$p_L(\mathbf{y}) := \operatorname*{argmin}_{\mathbf{x} \in \mathbb{R}^n} Q_L(\mathbf{x}, \mathbf{y}).$$  
ou, plus simplement :  
$$p_L(\mathbf{y}) = \underset{\mathbf{x}}{\operatorname*{argmin}} \left\{ g(\mathbf{x}) + \frac{L}{2} \left\| \mathbf{x} - \left( \mathbf{y} - \frac{1}{L} \nabla f(\mathbf{y}) \right) \right\|^2 \right\}.$$  
Soit :  
$$\boxed{p_L(\mathbf{y}) = \mathrm{prox}_{\frac{1}{L}g}\left(\mathbf{y} - \frac{1}{L}\nabla f(\mathbf{y})\right)}$$

### Algorithme

$$
\boxed{
\begin{array}{ll}
\textbf{ISTA with constant stepsize} & \\
\textbf{Input:} & L := L(f) \text{ – A Lipschitz constant of } \nabla f. \\
\textbf{Step 0.} & \text{Take } x_0 \in \mathbb{R}^n. \\
\textbf{Step k.} & (k \geq 1) \text{ Compute} \\
& \quad (3.1) \quad x_k = p_L(x_{k-1}).
\end{array}
}
$$

In [1]:
from linear_tools import ista

Le fichier \LASSO\Basis.ipynb présente un bon exemple d'utilisation de la méthode Lasso.

## Comparison

In [2]:
import numpy as np

import sys, pathlib
# You are in .../CAS LINEAIRE/OUTILS/.
project_root = pathlib.Path.cwd().parent.parent.parent.parent        # goes up to .../CAS LINEAIRE
sys.path.append(str(project_root))              # makes comp_translate visible

from comp_translate.SaveLoadTxt import dump_txt, load_txt

In [3]:
project_root

WindowsPath('c:/Users/Le R/Desktop/Code/Projets/Geneve/STAGE-GENEVE')

### Setup

In [40]:
n, p = 100, 50
sigma = 0.1
λ = 0.1
X, y = load_txt(project_root / "comp_translate/data/ISTA.txt")
y = y.reshape(-1, 1)

### Results

In [42]:
# Lipschitz constant of ∇f: L = ‖X‖₂²
L = np.linalg.norm(X, 2) ** 2

def f(b: np.ndarray) -> float:
    """0.5‖y – Xb‖²"""
    r = y - X @ b
    return 0.5 * np.linalg.norm(r)**2

def grad_f(b: np.ndarray) -> np.ndarray:
    """∇f(b) = –Xᵀ(y – Xb)"""
    return -X.T @ (y - X @ b)

def prox_l1(z: np.ndarray) -> np.ndarray:
    """Soft–thresholding (prox for λ‖·‖₁)."""
    return np.sign(z) * np.maximum(np.abs(z) - λ/L, 0.0)

beta0 = np.zeros((p,1))

In [43]:
beta_hat = ista(grad_f=grad_f,prox_g=prox_l1,x0=beta0,prox_g_args=(), L=L,max_iter=10000,tol=1e-9)

(50, 1)


In [44]:
f(beta_hat) + λ * np.linalg.norm(beta_hat, 1) 

np.float64(4.27951435542692)

In [45]:
beta_hat

array([[-0.22167347],
       [-0.81076479],
       [ 1.21973264],
       [ 2.1296557 ],
       [ 0.85821945],
       [-0.31706216],
       [ 0.64940299],
       [-0.27917048],
       [ 1.10513044],
       [-0.11074051],
       [ 1.37091546],
       [-0.2255344 ],
       [-0.46065175],
       [ 0.27874013],
       [-0.06087299],
       [-0.41398742],
       [ 2.03298921],
       [ 0.79580748],
       [ 0.6317235 ],
       [ 0.58165968],
       [ 0.1031761 ],
       [ 0.26391   ],
       [ 0.28993538],
       [-0.90545034],
       [ 1.90577112],
       [-0.67079618],
       [ 0.93406891],
       [ 0.6743858 ],
       [ 0.89318592],
       [-0.60058302],
       [-1.99923069],
       [-0.53729788],
       [-1.04430363],
       [-0.26719131],
       [ 0.45900135],
       [-0.14062524],
       [ 2.33126355],
       [ 0.66337133],
       [ 1.96720449],
       [-0.63909029],
       [ 1.26966542],
       [ 0.26028176],
       [ 2.00004169],
       [-0.27439617],
       [-1.11617976],
       [-1

## Amélioration avec le backtracking

Au lieu de fixer $L$, on commence avec une valeur initiale $L_0$ et on l'ajuste dynamiquement à chaque itération jusqu'à ce que la condition suivante soit satisfaite :
$$F(p_L(x)) \le Q_L(p_L(x), x)$$
où :
- $F(x) = f(x) + g(x)$,
- $Q_L(\cdot, x)$ est une approximation quadratique de $f$ autour de $x$, avec $L$ comme paramètre de courbure.

Le paramètre $\eta > 1$ est un facteur multiplicatif qui détermine comment $L$ est augmenté à chaque itération du backtracking. À chaque itération où la condition n'est pas satisfaite, on multiplie $L$ par $\eta$ ($L_{new} = \eta L_{old}$) jusqu'à ce que la condition soit vérifiée.

Typiquement, on choisit $\eta = 2$ ou $\eta = 1.5$. Un $\eta$ plus grand permet une adaptation plus rapide mais potentiellement moins précise de $L$, tandis qu'un $\eta$ plus proche de 1 donne une adaptation plus fine mais nécessite plus d'itérations.

$$
\boxed{
\begin{array}{ll}
\textbf{ISTA with backtracking} & \\
\textbf{Step 0.} & \text{Take } L_0 > 0,\ \eta > 1,\ \text{and } x_0 \in \mathbb{R}^n. \\
\textbf{Step k.} & (k \geq 1) \text{ Find the smallest nonnegative integers } i_k \text{ such that} \\
& \quad \bar{L} = \eta^{i_k} L_{k-1} \text{ and} \\
& \quad F(p_{\bar{L}}(x_{k-1})) \leq Q_{\bar{L}}(p_{\bar{L}}(x_{k-1}), x_{k-1}). \\
& \text{Set } L_k = \eta^{i_k} L_{k-1} \text{ and compute:} \\
(3.3) & x_k = p_{L_k}(x_{k-1}).
\end{array}
}
$$

In [ ]:
from linear_tools import ista_backtracking

Pareil le fichier \LASSO\Basis.ipynb présente un bon exemple d'utilisation de la méthode Lasso.